In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from netprop.data import Data
from netprop.dorm_model import DormModel
from netprop.model import Model

## Simulate Data

In [2]:
# simulate data set
np.random.seed(123)
p = [0.1, 0.6, 0.2, 0.1]
n = 1000
y = np.random.multinomial(n, p, size = 30)

In [3]:
df = pd.DataFrame({
    "obs": np.hstack([np.log((y[:10, 1])/(y[:10, 0])),
                      np.log((y[10:20, 1] + y[10:20, 2] + y[10:20, 3])/y[10:20, 0]),
                      np.log((y[20:30, 1] + y[20:30, 3])/(y[20:30, 0]))]),
    "obs_se": np.ones(y.shape[0])*1e-4,
    "ref_dorm": "A",
    "alt_dorm": ["B"]*10 + ["B_C_D"]*10 + ["B_D"]*10
})

# Config Model

In [4]:
data = Data.load(df,
                 obs="obs",
                 obs_se="obs_se",
                 ref_dorm="ref_dorm",
                 alt_dorm="alt_dorm",
                 dorm_separator="_")

In [10]:
dorm_models = [
    DormModel(name="A", covs=["intercept"], gprior={"intercept": [0.0, 1.0]}),
    DormModel(name="B", covs=["intercept"]),
    DormModel(name="C", covs=["intercept"]),
    DormModel(name="D", covs=["intercept"])
]

AttributeError: 'numpy.ndarray' object has no attribute 'items'

In [11]:
dorm_models[0].gprior

array([[ 0.],
       [inf]])

In [6]:
model = Model(data, dorm_models, gold_dorm="B")

## Fit Model

In [ ]:
model.fit_model()

## Predict Model and Create Draws

In [ ]:
model.predict(pd.DataFrame({"intercept": 1}, index=[0]))

In [ ]:
beta_samples = np.random.multivariate_normal(mean=model.beta, cov=model.beta_vcov, size=100)

In [ ]:
draws = np.vstack([
    model.predict(pd.DataFrame({"intercept": 1}, index=[0]), beta=beta)
    for beta in beta_samples
])

In [ ]:
fig, ax = plt.subplots();
ax.boxplot(draws);
ax.set_xticklabels(["A", "B", "C", "D"]);